### Definição das **classes de estados** e **conexões**

In [1]:

# Classe base
class TCPState:
    """
    Essa é uma classe abstrata que define as operações possíveis no protocolo TCP.
    """

    def open(self):
        """
        Solicita abertura de conexão.
        """
        raise NotImplementedError("Transição inválida para este estado.")
    
    def close(self):
        """
        Solicita encerramento da conexão.
        """
        raise NotImplementedError("Transição inválida para este estado.")
    
    def send(self):
        """
        Envia bytes.
        """
        raise NotImplementedError("Transição inválida para este estado.")
    def receive(self):

        """
        Recebe bytes
        """
        raise NotImplementedError("Transição inválida para este estado.")


# Classe que gerencia a máquina de estados.
class TCPConnection:
    """
    A classe vai encapsular o estado atual e delega operações dos estados
    """

    def __init__(self):
        self.state = ClosedState()  # Inicializa Estado 1

    def open(self):
        self.state = self.state.open() 
    def close(self):
        self.state = self.state.close()
    def send(self):
        self.state = self.state.send()
    def receive(self):
        self.state = self.state.receive()




# Classe do estado 1
class ClosedState(TCPState):
    """
    Estado inicial e final do protocolo TCP. Representa uma conexão fechada.
    """

    def open(self):
        """
         Envia um segmento SYN para iniciar a conexão e transita para SYN_SENT.
        """
        print("Transição: CLOSED → SYN_SENT")
        return SynSentState()

# Classe do estado 2
class SynSentState(TCPState):
    """
    Estado em que um segmento SYN foi enviado 
    e o cliente aguarda uma resposta (SYN+ACK).
    """
    def receive(self):

        """
        Recebe um segmento SYN+ACK do servidor, estabelecendo a conexão. 
        Transita para ESTABLISHED.
        """
        print("Transição: SYN_SENT → ESTABLISHED (SYN+ACK recebido)")
        return EstablishedState()


# Classe do estado 3 
class EstablishedState(TCPState):
    """
    Estado principal, onde a conexão está ativa e dados podem ser enviados e recebidos.
    """

    def send(self):
        """
        Envia dados. Permanece no estado ESTABLISHED.
        """
        print("Ação: Dados enviados no estado ESTABLISHED")
        return self
    
    def receive(self):
        """
        Recebe dados. Permanece no estado ESTABLISHED.
        """
        print("Ação: Dados recebidos no estado ESTABLISHED")
        return self
    

    def close(self):
        """
        Inicia o encerramento da conexão enviando um segmento FIN. 
        Transita para FIN_WAIT_1.
        """
        print("Transição: ESTABLISHED → FIN_WAIT_1")
        return FinWait1State()


# Classe do estado 4
class FinWait1State(TCPState):
    """
    Estado em que o segmento FIN foi enviado e o cliente aguarda o ACK correspondente.
    """

    def receive(self):
        """
        Recebe o ACK do segmento FIN enviado. 
        Transita para TIME_WAIT
        """
        print("Transição: FIN_WAIT_1 → TIME_WAIT (ACK para FIN recebido)")
        return TimeWaitState()


# Classe do estado 5
class TimeWaitState(TCPState):
    def close(self):
        """
        Estado que aguarda para garantir que todos os pacotes em
        trânsito tenham sido recebidos antes de fechar a conexão.
        """

        print("Transição: TIME_WAIT → CLOSED (Tempo esgotado)")
        return ClosedState()




# Testes da máquina de estado finita

In [5]:
# Simulação do ciclo de vida TCP
connection = TCPConnection()
print("[Estado Inicial: CLOSED]")


[Estado Inicial: CLOSED]


In [4]:

# Estabelecendo a conexão
connection.open()  # CLOSED → SYN_SENT
connection.receive()  # SYN_SENT → ESTABLISHED


Transição: CLOSED → SYN_SENT
Transição: SYN_SENT → ESTABLISHED (SYN+ACK recebido)


In [ ]:

# Trocando dados
connection.send()  # Envia dados
connection.receive()  # Recebe dados


In [ ]:

# Encerrando a conexão
connection.close()  # ESTABLISHED → FIN_WAIT_1
connection.receive()  # FIN_WAIT_1 → TIME_WAIT
connection.close()  # TIME_WAIT → CLOSED

print("[Conexão encerrada: CLOSED]")
